<a href="https://colab.research.google.com/github/basselkassem/monte-carlo-search-tree/blob/master/MCTreeSearsh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gym
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
from queue import Queue, LifoQueue
from copy import copy
import numpy as np
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        #print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

In [11]:
env = gym.make('Taxi-v3').env
init_state = env.reset()
env.render()

action_num = env.action_space.n
state_num = env.observation_space.n

print('Action space:', action_num)
print('Observation space:', state_num)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action space: 6
Observation space: 500


In [12]:
state_id = 328
env.s = state_id
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [13]:
env.P[state_id]
#{action: [(probability, nextstate, reward, done)]}

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

# Random policy

In [18]:
env.reset()
env.render()

is_done = False
total_reward, penalty, epochs = 0, 0, 0
while not is_done:
  action = env.action_space.sample()

  new_state, reward, is_done, info = env.step(action)

  total_reward += reward
  if reward == -10:
    penalty += 1
  epochs += 1

env.render()
print('Timesteps taken:', epochs)
print('Penalty:', penalty)
print('total_reward:', total_reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Timesteps taken: 3547
Penalty: 1169
total_reward: -14047


In [9]:
  print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

Timestep: 48
State: 284
Reward: -10


KeyboardInterrupt: ignored

#Planning with BFS & DFS

In [0]:
class Node:

  def __init__(self, parent, env):
    self.visited_times = 0
    self.parent = parent
    self.children = set()
    self.observation = None
    self.reward = 0
    self.is_done = False
    self.env_space = None
    self.init_env_space(env)
  
  def init_env_space(self, env):
    self.env_space = copy.deepcopy(env)

  def is_terminal(self):
    return self.is_done
  
  def get_env_space(self, action, parent_env):
    self.observation, self.reward, self.is_done, _ = parent_env.step(action)
    return parent_env

  def expand(self):
    #print(self.env_space)
    for action in range(0, env.action_space.n):
      parent_env = copy.deepcopy(self.env_space)
      child_env_space = self.get_env_space(action, parent_env)
      child_env_space.render()
      print(action, self.is_done)
      child = Node(self, child_env_space)
      self.children.add(child)



In [0]:
def planning_BFS(root):
  total_reward, penalty, epochs = 0, 0, 0
  queue = Queue()
 
  root.visited_times += 1
  queue.put(root)

  while not queue.empty():
    current_node = queue.get()
    total_reward += current_node.reward
    if current_node.reward == -10:
      penalty += 1
    epochs += 1
    current_node.expand()
    if current_node.is_done == True:
      return {
          'node': current_node,
          'penalty': penalty,
          'reward': total_reward,
          'time_steps': epochs,
      }
    for child_node in current_node.children:
      if child_node.visited_times == 0:
        child_node.visited_times += 1
        queue.put(child_node)

      

In [0]:
def planning_DFS(root):
  total_reward, penalty, epochs = 0, 0, 0
  stack = LifoQueue()

  root.visited_times += 1
  stack.put(root)

  while not stack.empty():
    current_node = stack.get()
    total_reward += current_node.reward
    if current_node.reward == -10:
      penalty += 1
    epochs += 1
    current_node.expand()
     if current_node.is_done == True:
      return {
          'node': current_node,
          'penalty': penalty,
          'reward': total_reward,
          'time_steps': epochs,
      }
    for child_node in current_node.children:
      if child_node.visited_times == 0:
        child_node.visited_times += 1
        stack.put(child_node)
      

In [0]:
state = env.encode(1, 0, 1, 0)
state = 499
env.s = state
env.render()

#o, r, is_done, _ = env.step(0)

#o, r, is_done, _ = env.step(3)
print(env.s)
#o, r, is_done, _ = env.step(5)
print(is_done)
env.render()


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (South)
499
False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (South)


In [0]:
state = 499

In [0]:
#env.s = state
env.reset()
env.render()

root = Node(None, None)
root.init_env_space(env)
termianl = planning_BFS(root)

env.s = termianl['node'].observation
env.render()
print('Timesteps taken:', termianl['time_steps'])
print('Penalty:', termianl['penalty'])
print('total_reward:', termianl['reward'])


Streaming output truncated to the last 5000 lines.
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
5 False
-----<< False
>>----- False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
0 False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
1 False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
2 False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
3 False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
4 False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
5 False
-----<< False
>>----- False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
0 False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
1 False
+---------+
|R: | : 

In [0]:
def construct_path(node):
  solutions = []
  temp_node = node['node']
  while temp_node.parent != None:
    solutions.append(temp_node)
    temp_node = temp_node.parent
  return reversed(solutions)

solutions = construct_path(termianl)
for sol in solutions:
  #clear_output(wait=True)
  env.s = sol.observation
  env.render()
  #sleep(.1)

# Learning Action-value Function

In [0]:
gamma = 0.6
alpha = 0.1
epsilon = 0.1

def epsilon_greedy(q_values, state, epsilon):
  action = None
  if np.random.uniform(0, 1) < epsilon:
    action = env.action_space.sample()
  else:
    action = np.argmax(q_values[state])
  return action

def q_learning(iter_num):
  q_values = np.zeros([state_num, action_num])
  print(q_values.shape)
  for i in range(iter_num):
    state = env.reset()
    done = False
    while not done:
      action = epsilon_greedy(q_values, state, epsilon)
      next_state, reward, done, info = env.step(action)
      q_values[state, action] += alpha * (reward + gamma * np.max(q_values[next_state]) - q_values[state, action])
      state = next_state
    if i % 100 == 0:
      clear_output(wait=True)
      print(f"Episode: {i}")
  print("Training finished.\n")
  return q_values


In [0]:
q_policy = q_learning(10000)


Episode: 9900
Training finished.



In [0]:
is_done = False
total_reward, penalty, epochs = 0, 0, 0
frames = []
state = env.reset()
env.render()
while not is_done:
  action = np.argmax(q_policy[state])
  state, reward, is_done, info = env.step(action)

  total_reward += reward
  if reward == -10:
    penalty += 1
  epochs += 1
  frames.append({
        'frame': env.render(mode='ansi'),
        'state': observation,
        'action': action,
        'reward': reward
        }
  )

print('Timesteps taken:', epochs)
print('Penalty:', penalty)
print('total_reward:', total_reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Timesteps taken: 13
Penalty: 0
total_reward: 8


In [0]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 13
State: 334
Action: 5
Reward: 20


# Integrating Learning and Planning

In [0]:
class TNode:

  def __init__(self, env_space, parent = None):
    self.visiting_times = 0
    self.parent = parent
    self.value_sum = 0
    self.children = set()
    self.env_space = env_space
    self.is_done = False
    self.observation = None
    self.reward = 0
    self.action = None
    self.untried_actions = [action for action in range(action_num)]
  
  def is_leaf(self):
    return len(self.children) <= 0
  
  def is_root(self):
    return self.parent == None
  
  def is_termial(self):
    return self.is_done
  
  def compute_mean_value(self):
    if self.visiting_times == 0:
      return 0
    return self.value_sum / self.visiting_times  

  def compute_score(self, scale = 10, max_score = 10e100):
    if self.visiting_times == 0:
      return max_score
    parent_visiting_times = self.parent.visiting_times
    ucb = 2 * np.sqrt(np.log(parent_visiting_times / self.visiting_times))
    return self.compute_mean_value() + scale * ucb

  def select_best_leaf(self):
    if self.is_leaf():
      return self
    children = list(self.children)
    #print(len(children))
    scores = [child.compute_score() for child in children]
    child_index = np.argmax(scores)
    best_child = children[child_index]
    return best_child.select_best_leaf()
  
  def expand(self):
    if self.is_termial():
      return None
    for action in range(action_num):
      env_space = copy(self.env_space)
    #action = env_space.action_space.sample()
    #action = self.untried_actions.pop()
    #print(action, len(self.untried_actions))
      self.observation, self.reward, self.is_done,_ = env_space.step(action)
      node = TNode(env_space, self)
      node.action = action
      self.children.add(node)
    return self.select_best_leaf()
  
  def rollout(self, t_max = 10**8):
    env_space = copy(self.env_space)
    rollout_return = 0
    for t in range(t_max):
      #action = np.argmax(q_policy[env_space.s])
      action = env_space.action_space.sample()
      obs, reward, done, _ = env_space.step(action)
      rollout_return += reward
      if done:
        break

    return rollout_return

  def propagate(self, child_value):
    node_value = self.reward + child_value
    self.value_sum += node_value
    self.visiting_times += 1
    if not self.is_root():
      return self.parent.propagate(node_value)
  
  def safe_delete(self):
    del self.parent
    for child in self.children:
      child.safe_delete()
      del child


In [0]:
def planning_MCT(root, n_iter = 10):
  for itr in range(n_iter):
    node = root.select_best_leaf()
    if node.is_done:
      break
      #node.propagate(0)
    #else:
    selected_node = node.expand()
    returned_value = node.rollout()
    node.propagate(returned_value)

In [53]:
state_id = 171
env.s = state_id
n_iter = 10**12
#env.reset()
print(env.s)

env.render()
root = TNode(env)
planning_MCT(root, n_iter = n_iter)
is_done = False
total_reward, penalty, epochs = 0, 0, 0
frames = []
test_env = root.env_space
Tnode = root
#while not is_done:
for i in range(50):
  children = list(Tnode.children)
  best_leaf_node = np.argmax([node.compute_score() for node in children])
  best_child = children[best_leaf_node]
  observation, reward, is_done, info = test_env.step(best_child.action)

  total_reward += reward
  if reward == -10:
    penalty += 1
  epochs += 1
  frames.append({
        'frame': env.render(mode='ansi'),
        'state': observation,
        'reward': reward
        }
    )
  #for child in root.children:
    #if child != best_child:
      #child.safe_delete()
  Tnode = best_child
  if Tnode.is_leaf():
    planning_MCT(Tnode, n_iter = n_iter)

print('Timesteps taken:', epochs)
print('Penalty:', penalty)
print('total_reward:', total_reward)
Tnode.env_space.render()
print(Tnode.env_space.s)

171
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


KeyboardInterrupt: ignored

In [52]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | :_|
|Y| : |B: |
+---------+
  (North)

Timestep: 100
State: 399
Reward: -1


# MCTS

In [0]:
class Node:

  def __init__(self, env, parent = None):
    self.state = env
    self.parent = parent
    self.children = []
    self.untried_actions = [action for action in range(action_num)]
    self.visiting_times = 0
    self.q = 0
    self.is_done = False
    self.observation = None
    self.reward = 0
    self.action = None

  def is_fully_expanded(self):
    return len(self.untried_actions) == 0

  def is_terminal_node(self):
    return self.is_done

  def compute_mean_value(self):
    if self.visiting_times == 0:
      return 0
    return self.q / self.visiting_times

  def compute_score(self, scale = 10, max_score = 10e100):
    if self.visiting_times == 0:
      return max_score
    parent_visiting_times = self.parent.visiting_times
    ucb = 2 * np.sqrt(np.log(parent_visiting_times) / self.visiting_times)
    result = self.compute_mean_value() + scale * ucb
    return result

  def best_child(self):
    scores = [child.compute_score() for child in self.children]
    child_index = np.argmax(scores)
    return self.children[child_index]

  def expand(self):
    action = self.untried_actions.pop()
    next_state = copy(self.state)
    self.observation, self.reward, self.is_done,_ = next_state.step(action)
    child_node = Node(next_state, parent = self)
    child_node.action = action
    self.children.append(child_node)
    return child_node
  
  def rollout_policy(self, state):
    return state.action_space.sample()
  
  def rollout(self, t_max = 10**8):
    state = copy(self.state)
    rollout_return = 0
    gamma = 0.6
    done = False
    while not done:
      action = self.rollout_policy(state)
      obs, reward, done, _ = state.step(action)
      rollout_return += gamma * reward
      if done:
        break

    return rollout_return

  def backpropagate(self, child_value):
    node_value = self.reward + child_value
    self.q += node_value
    self.visiting_times += 1
    if self.parent:
      return self.parent.backpropagate(node_value)


class MonteCarloTreeSearch(object):
  def __init__(self, node):
    self.root = node

  def best_action(self, simulations_number):
    for _ in range(0, simulations_number):
      v = self._tree_policy()
      reward = v.rollout()
      v.backpropagate(reward)
    return self.root.best_child()

  def _tree_policy(self):
    current_node = self.root
    while not current_node.is_terminal_node():
      if not current_node.is_fully_expanded():
        return current_node.expand()
      else:
        current_node = current_node.best_child()
    return current_node

In [25]:
env.reset()
env.render()

n_simulation = 10**4
root = Node(env)
is_done = False
total_reward, penalty, epochs = 0, 0, 0

while not is_done:
  mcts = MonteCarloTreeSearch(root)
  best_child = mcts.best_action(n_simulation)
  new_state, reward, is_done, info = env.step(best_child.action)
  total_reward += reward
  if reward == -10:
    penalty += 1
  epochs += 1
  root = best_child

env.render()
print('Timesteps taken:', epochs)
print('Penalty:', penalty)
print('total_reward:', total_reward)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Timesteps taken: 34
Penalty: 13
total_reward: -130
